In [1]:
!ls

 cf_model.py	  main.ipynb					  todo.txt
 config.py	  menv						  todo.txt.lock
 dataset.py	  movie.py					  train.ipynb
 db.ipynb	  __pycache__					  user.py
 done.txt	  q.py						  wandb
 fdstests.ipynb   README.md					  web_spider.py
 filemanager.py  'Spider-1_doing_Spider-Man: Far From Home.txt'
 letterboxd.db	 'Spider-4_doing_Spider-Man: Far From Home.txt'


In [2]:
import sqlite3

In [3]:
def create_db(db_name):

    # Create a connection to a database file (it will be created if it doesn't exist)
    conn = sqlite3.connect(db_name+'.db')
    
    # Create a cursor object using the connection
    cursor = conn.cursor()
    
    # Example: Create a table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Genres (
            genre_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE
        );
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Directors (
            director_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE
        );
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Actors (
            actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE
        );
    ''')

#            movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
    # Example: Create a table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Movies (
            movie_id CHAR(36) PRIMARY KEY,
            title TEXT NOT NULL,
            release_date DATE,
            director_id INTEGER,
            country_id INTEGER,
            FOREIGN KEY (country_id) REFERENCES Countries(country_id)
            FOREIGN KEY (director_id) REFERENCES Directors(director_id)
        );

    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Countries (
            country_id INTEGER PRIMARY KEY AUTOINCREMENT,
            country_name VARCHAR(255) NOT NULL
        );
    ''')


    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MoviesGenres (
            movie_id CHAR(36),
            genre_id INTEGER,
            PRIMARY KEY (movie_id, genre_id),
            FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
            FOREIGN KEY (genre_id) REFERENCES Genres(genre_id)
        );

    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MoviesActors(
            movie_id CHAR(36),
            actor_id INTEGER,
            PRIMARY KEY (movie_id, actor_id),
            FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
            FOREIGN KEY (actor_id) REFERENCES Actors(actor_id)
        );

    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Users (
            user_id CHAR(36) PRIMARY KEY,
            name TEXT NOT NULL
            
        );
    ''')

    cursor.execute('''
        CREATE TABLE Reviews (
            review_id INTEGER PRIMARY KEY AUTOINCREMENT,
            movie_id CHAR(36),
            user_id CHAR(36),
            review_text TEXT,
            rating DECIMAL NOT NULL,
            review_date DATE,
            FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
            FOREIGN KEY (user_id) REFERENCES Users(user_id)
        );
    ''')

    # Commit the changes
    conn.commit()
    
    # Close the connection
    conn.close()
    
    print("SQLite database and table created locally.")

In [4]:
def drop_tables(db_name):
    # Create a connection to a database file (it will be created if it doesn't exist)
    conn = sqlite3.connect(db_name+'.db')
    
    # Create a cursor object using the connection
    cursor = conn.cursor()

    table_names = ['Movies', 'Users', 'Reviews', 'Actors','Directors','Genres', 'MoviesActors','MoviesGenres']
    for table_name in table_names:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    conn.commit()
    # Close the connection
    conn.close()


In [5]:
def show_tables():

    
    conn = sqlite3.connect(db_name+'.db')
    
    # Create a cursor object using the connection
    cursor = conn.cursor()
    
    # (Your existing table creation commands go here)
    
    # After creating tables, query the sqlite_master table to list them
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()

    
    # Commit the changes
    conn.commit()
    
    # Close the connection
    conn.close()
    
    # Print the names of all tables created
    
    for table in tables:
        print(table[0])


In [6]:
# def get_all_table(args*):

#     conn = sqlite3.connect('./movusers.db')
    
#     # Create a cursor object using the connection
#     cursor = conn.cursor()
    
#     # (Your existing table creation commands go here)
    
#     # After creating tables, query the sqlite_master table to list them
#     query = "SELECT " + args[0] + " FROM " + args[1] + " WHERE " + args[2] + " == " + args[3]
#     cursor.execute(query)
#     tables = cursor.fetchall()

    
#     # Commit the changes
#     conn.commit()
    
#     # Close the connection
#     conn.close()
    

In [7]:
def add_movie(movie, db_name):
    
    conn = sqlite3.connect(db_name + '.db')
    cursor = conn.cursor()

    # Insert genres
    print(movie['genres'])
    for genre in movie['genres']:
        genre_exists = cursor.execute("SELECT 1 FROM Genres WHERE name = ?", (genre,)).fetchone()
        
        if not genre_exists:
            # print(genre)
            cursor.execute("INSERT INTO Genres (name) VALUES(?)", (genre,))

    # Insert country
    # print(movie['country'])
    country_exists = cursor.execute("SELECT 1 FROM Countries WHERE country_name = ?", (movie['country'],)).fetchone()
    if not country_exists:
        
        cursor.execute("INSERT INTO Countries (country_name) VALUES(?)", (movie['country'],))
    
    # Insert movie
    cursor.execute("INSERT INTO Movies (movie_id, title, release_date, director_id) VALUES (?, ?, ?, ?)", 
                   (movie['id'], movie['title'], movie['release date'], movie['director id']))

    # Assuming movie['genres'] contains genre names, you must fetch each genre_id from Genres table
    # and then insert each movie-genre relationship
    for genre in movie['genres']:
        cursor.execute("SELECT genre_id FROM Genres WHERE name = ?", (genre,))
        genre_id = cursor.fetchone()[0]
        cursor.execute("INSERT INTO MoviesGenres (movie_id, genre_id) VALUES (?, ?)", (movie['id'], genre_id))

    conn.commit()
    conn.close()


In [8]:
def add_user(user,db_name):


    conn = sqlite3.connect(db_name+'.db')
    
    # Create a cursor object
    cursor = conn.cursor()
    # cursor.execute("INSERT INTO Users (name) VALUES ('John Doe')")

    cursor.execute("INSERT INTO Users (user_id,name) VALUES (?,?)", (user.user_id,user.name))
    # Commit the changes
    conn.commit()
    conn.close()
    

In [9]:
def add_movie_review(mr,db_name):

        # {'title':'asdfas',
        # 'movie_id':movie['id'],
        # 'user_id': juser.user_id
        # 'details':{
        #     'ratings':7,
        #     'author':'ccharlesdicks',
        #     'content':'it was good'
        # }

    #     cursor.execute('''
    #     CREATE TABLE Reviews (
    #         review_id INTEGER PRIMARY KEY AUTOINCREMENT,
    #         movie_id INTEGER,
    #         user_id INTEGER,
    #         review_text TEXT,
    #         rating DECIMAL,
    #         review_date DATE,
    #         FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
    #         FOREIGN KEY (user_id) REFERENCES Users(user_id)
    #     );
    # ''')
    # dict_keys(['title', 'movie_id', 'ratings', 'author', 'date', 'content', 'user_id'
        conn = sqlite3.connect(db_name+'.db')
        
        # Create a cursor object
        cursor = conn.cursor()

        # print(mr.keys())
        cursor.execute("INSERT INTO Reviews (movie_id,user_id,review_text,rating,review_date) VALUES (?,?,?,?,?)", (mr['movie_id'],mr['user_id'],mr['content'],mr['ratings'],mr['date']))
        conn.commit()
        conn.close()

In [10]:
def display_data(db_name):
    conn = sqlite3.connect(db_name+'.db')
    cursor = conn.cursor()
    
    # Display some users
    cursor.execute("SELECT * FROM Users")
    users = cursor.fetchall()
    print("Users:")
    for user in users:
        print('thisuser',user)
    
    # Display some movies
    cursor.execute("SELECT * FROM Movies")
    movies = cursor.fetchall()
    print("\nMovies:")
    for movie in movies:
        print(movie)

    # Display some movies
    cursor.execute("SELECT * FROM Reviews")
    reviews = cursor.fetchall()
    print("\nReviews:")
    for review in reviews:
        print(review)
    
    conn.close()


In [11]:
def display_size(db_name):
    conn = sqlite3.connect(db_name+'.db')
    cursor = conn.cursor()
    
    # Display some users
    cursor.execute("SELECT * FROM Users")
    users = cursor.fetchall()
    print("Users:",len(users))

    
    # Display some movies
    cursor.execute("SELECT * FROM Movies")
    movies = cursor.fetchall()
    print("\nMovies:",len(movies))

    # Display some movies
    cursor.execute("SELECT * FROM Reviews")
    reviews = cursor.fetchall()
    print("\nReviews:",len(reviews))
    conn.close()


In [12]:
def insert_data(db_name):
    conn = sqlite3.connect(db_name+'.db')
    cursor = conn.cursor()
    
    # Inserting a director
    cursor.execute("INSERT INTO Directors (name) VALUES ('Christopher Nolan')")
    director_id = cursor.lastrowid

    # Inserting a genre
    cursor.execute("INSERT INTO Genres (name) VALUES ('Sci-Fi')")
    genre_id = cursor.lastrowid
    
    # Inserting a movie
    cursor.execute("INSERT INTO Movies (title, release_date, director_id, overview) VALUES (?, ?, ?, ?)", 
                   ("Inception", "2010-07-16", director_id, "A thief who steals corporate secrets..."))
    movie_id = cursor.lastrowid
    
    # Inserting a user
    cursor.execute("INSERT INTO Users (name) VALUES ('John Doe')")
    user_id = cursor.lastrowid
    
    # Assume more inserts here for Actors, MoviesActors, Reviews, etc.
    
    conn.commit()
    conn.close()
    print("Data inserted successfully.")


In [13]:
def get_table_values(table_name,db_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name+'.db')
    cursor = conn.cursor()
    
    # Execute a SELECT query to retrieve all values from the specified table
    query = "SELECT * FROM " + table_name
    print(query)
    cursor.execute(query)

    # Fetch all the rows from the result set
    rows = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return rows

In [14]:
def main():
    pass
    # drop_tables()
    # print('dropped')
    # create_db()
    # show_tables()

    # #populate tables.... will need to be done by program....
    # from user import User

    
    # juser = User('jow')
    # print(vars(juser))  # This will show you the attributes of the user object

    # add_user(juser)
    # from datetime import datetime
    # movie = {

    #     'id':1324,
    #     'title':'jp',
    #     'release_date':datetime(2022,2,1),
    #     'director_id':1,
    #     'overview':'asdfasddfaf'
    # }

    # add_movie(movie)
    
    # # review = {'check':''}
    # # review['title'] = title_div[0].text
    # # review['movie_id'] = movie_id
    # # print(title_div[0].text)
    # # # Step 3: Retrieve and print the text for each child element
    # # details = {0:'ratings',1:'author',2:'content',3:'check'}
    # # count = 0




    # movie_review = {

    #     'title':'asdfas',
    #     'movie_id':movie['id'],
    #     'user_id': juser.user_id,
    #     'details':{
    #         'rating':7,
    #         'author':'ccharlesdicks',
    #         'votes':100,
    #         'content':'it was good',
    #         'date': datetime(2022,2,1)
    #     }
        
    # }

    # add_movie_review(movie_review)

    # print(show_tables())
    # print(display_data())

    

In [15]:
# main()

In [16]:
# print(show_tables())
# print(display_data())